## 对函数$y=2\mathbf {x}^\mathrm T\mathbf x$关于列向量$\mathbf x$求导

In [2]:
import torch

In [3]:
x = torch.arange(4.0)
x.requires_grad_(True)
print(x)


tensor([0., 1., 2., 3.], requires_grad=True)


In [5]:
y = torch.dot(x, x)  # y = x_1^2 + x_2^2 + x_3^2 + x_4^2
print(y)

tensor(14., grad_fn=<DotBackward0>)


In [6]:
y.backward()
print(x.grad)  # x.grad = 2 * x

tensor([0., 2., 4., 6.])


## 对函数$y=\sum(\mathbf x)$关于列向量$\mathbf x$求导

In [19]:
x.grad.zero_()
y = x.sum()  # y = x_1 + x_2 + x_3 + x_4
y.backward()
print(x.grad)

tensor([1., 1., 1., 1.])


## 深度学习中 ，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [22]:
x.grad.zero_()
y = x * x  # y = tensor(0., 1., 4., 9.)
# y.backward()  # RuntimeError: grad can be implicitly created only for scalar outputs
y.sum().backward()  # y.sum() = x_1^2 + x_2^2 + x_3^2 + x_4^2
print(x.grad)

tensor([0., 2., 4., 6.])


## 将某些计算移动到记录的计算图之外

In [31]:
x.grad.zero_()
y = x * x  # y = tensor(0., 1., 4., 9.)
z = y.detach() * x  # z = tensor(0.*x_1, 1.*x_2, 4.*x_3, 9.*x_4)
z.sum().backward()  # z.sum() = 0.*x_1 + 1.*x_2 + 4.*x_3 + 9.*x_4
print(x.grad)

tensor([0., 1., 4., 9.])


## 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [10]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

x.grad.zero_()
y = f(x)  # y = x_1^k + x_2^k + x_3^k + x_4^k
print(x)
print(y)
y.sum().backward()
print(x.grad)

tensor([0., 1., 2., 3.], requires_grad=True)
tensor([   0.,  512., 1024., 1536.], grad_fn=<MulBackward0>)
tensor([512., 512., 512., 512.])
